In [1]:
import os
import torch
from datasets import load_dataset
from google.cloud import translate_v2 as translate
from concurrent.futures import ThreadPoolExecutor
gcp_project_id = "artful-tractor-338209"

C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\karti\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Create a Service Account:
In the Cloud Console, navigate to "IAM & Admin" > "Service accounts." Click "Create Service Account." Enter a name for service account, select the role(s) that service account needs (Cloud Translation API Editor), and click "Continue."

## Generate Key:
After creating the service account, click on it in the "Service accounts" page. Navigate to the "Keys" tab. Click on "Add Key" and choose "JSON." This will download a JSON key file containing the necessary credentials.

In [2]:
# give path to json key file

credential_path = "C:/Users/karti/Desktop/ComputerScience/internships/Nirant2/artful-tractor-338209-61f5638878ce.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

In [32]:
possible_configs = [
#     # "main",
#     # "socratic",
    'ARC-Challenge',
    'ARC-Easy'
]
dataset = []
for config in possible_configs:
    dataset_slice = load_dataset("ai2_arc", config,ignore_verifications=True)
    dataset.append(dataset_slice)
    


Found cached dataset parquet (C:/Users/karti/.cache/huggingface/datasets/parquet/ARC-Challenge-6372c4cac1109a78/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset parquet (C:/Users/karti/.cache/huggingface/datasets/parquet/ARC-Challenge-6372c4cac1109a78/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
# dataset
for g in dataset:
    print(g['train'])
    print(g['test'])
    print(g['validation'])

Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3370
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3548
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 869
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3370
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 3548
})
Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 869
})


In [3]:
def translate_text( text: str,target='hi') -> dict:
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)


    return result["translatedText"]
translate_text("How are you kartik")

'कार्तिक आप कैसे हैं?'

In [40]:
%%time

translate_text("{'text': ['dry palms', 'wet palms', 'palms covered with oil', 'palms covered with lotion'], 'label': ['A', 'B', 'C', 'D']}")
# translate_text("Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?	")
# translate_text("How many clips did Natalia sell in May? ** Natalia sold 48/2 = <<48/2=24>>24 clips in May. How many clips did Natalia sell altogether in April and May? ** Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May. #### 72")


CPU times: total: 15.6 ms
Wall time: 934 ms


'{&#39;पाठ&#39;: [&#39;सूखी हथेलियाँ&#39;, &#39;गीली हथेलियाँ&#39;, &#39;तेल से ढकी हथेलियाँ&#39;, &#39;लोशन से ढकी हथेलियाँ&#39;], &#39;लेबल&#39;: [&#39;ए&#39;, &#39;बी&#39;, &#39;सी&#39;, &#39;डी&#39; &#39;]}'

In [7]:
print(dataset)

[DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
}), DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})]


In [8]:
import os

num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

Number of CPU cores: 16


In [45]:
%%time
import time

for i in range(len(possible_configs)):
    for set in dataset[i]:
        set_list = []
        columns = ['question','choices']
        
        for col in columns:
            values = [str(item[col]) for item in dataset[i][set]]
            print(values[1:2])

            if __name__ == '__main__':
                result =[]
                with ThreadPoolExecutor(max_workers=16) as exe:
                    # Maps the method with a list of values.
                    result = list(exe.map(translate_text,values[1:2]))
                    print(result)
                    # batch_size = 16
                    # for j in range(0,48,batch_size):
                    #     j_end = min(j + batch_size, 48)
                    #     res = list(exe.map(translate_text,values[j:j_end]))
                    #     time.sleep(2)
                    #     result.extend(res)
                    
            set_list.append(result)
        
        current_directory = os.getcwd()
        # Specify the path of the 'config' folder
        config_folder_path = os.path.join(current_directory, possible_configs[i])
        # Create the 'config' folder
        os.makedirs(config_folder_path, exist_ok=True)       
    
        import csv
        # Transpose the 2D list
        transposed_data = list(map(list, zip(*set_list)))
        path = os.path.join(possible_configs[i], f'{set}.csv')
        with open(path, 'w', encoding='utf-8') as f:
            # using csv.writer method from CSV package
            write = csv.writer(f)
            # write.writerow(columns)
            write.writerows(transposed_data)

['Which of the following statements best explains why magnets usually stick to a refrigerator door?']
['निम्नलिखित में से कौन सा कथन सबसे अच्छी तरह से बताता है कि चुंबक आमतौर पर रेफ्रिजरेटर के दरवाजे से क्यों चिपकते हैं?']
["{'text': ['The refrigerator door is smooth.', 'The refrigerator door contains iron.', 'The refrigerator door is a good conductor.', 'The refrigerator door has electric wires in it.'], 'label': ['A', 'B', 'C', 'D']}"]
['{&#39;टेक्स्ट&#39;: [&#39;रेफ्रिजरेटर का दरवाजा चिकना है।&#39;, &#39;रेफ्रिजरेटर के दरवाजे में लोहा है।&#39;, &#39;रेफ्रिजरेटर का दरवाजा एक अच्छा कंडक्टर है।&#39;, &#39;रेफ्रिजरेटर के दरवाजे में बिजली के तार हैं।&#39;], &#39;लेबल &#39;: [&#39;ए बी सी डी&#39;]}']
['High-pressure systems stop air from rising into the colder regions of the atmosphere where water can condense. What will most likely result if a high-pressure system remains in an area for a long period of time?']
['उच्च दबाव प्रणालियाँ हवा को वायुमंडल के ठंडे क्षेत्रों में जाने से रोकती है